In [1]:
! ls Datasets/new_test

Fire  NoFire


In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [3]:
DATADIR = 'Datasets/new_test'
CATEGORIES = ['Fire', 'NoFire']

In [4]:
IMG_SIZE = 64
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data

In [5]:
training_data = create_training_data()


100%|██████████| 107/107 [00:02<00:00, 43.68it/s]


In [6]:
import random

print(len(training_data))
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

226
0
0
0
0
0
0
1
0
1
1


In [7]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
X.shape[1:]

(64, 64, 3)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import time


model=load_model(r'TrainedModels/Fire-64x64-color-v2.model')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [9]:
res = model.predict(X).tolist()
for i in range(len(res)):
    res[i] = res[i][0]

In [10]:
tp = 0
tn = 0
fp = 0
fn = 0

for i in range(len(res)):
    if Y[i] == 0:   # Actual Positive
        if res[i] <= 0.5:  # True Positive
            tp += 1
        else:             # False Negative
            fn += 1
    else: # Actual Negative
        if res[i] <= 0:  # False Positive
            fp += 1
        else:             # True Negative
            tn += 1
        

In [11]:
print(tp, tn, fp, fn)

105 107 0 14


In [12]:
precision = tp / (tp + fp)
recall = tp/(tp+fn)
f1_score = 2*(recall * precision) / (recall + precision)
print(precision, recall, f1_score)


1.0 0.8823529411764706 0.9375


In [13]:
model.evaluate(X, Y)

226/226 [==============================] - 0s 1ms/sample - loss: 0.4545 - acc: 0.8894


[0.45447546189683863, 0.8893805]